In [1]:
import pickle as pic
import numpy as np
import pandas as pd
from sklearn.preprocessing import MultiLabelBinarizer
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import CountVectorizer
import nltk.data
from sklearn.model_selection import train_test_split

from __future__ import print_function

import keras
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import SimpleRNN
from keras.optimizers import SGD

import matplotlib
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style('white')

%matplotlib inline
nltk.download('stopwords')

Using TensorFlow backend.


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Keenan\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [2]:
def save_obj(obj, name):
    with open(name, 'wb') as f:
        pic.dump(obj, f, pic.HIGHEST_PROTOCOL)
def load_obj(name):
    with open(name, 'rb') as f:
        return pic.load(f, encoding='latin1')

In [3]:
ten_k = load_obj('../data/tmdb_df_10k.pickle')

In [4]:
def df_overview_clean(df):
    ret_df = df.copy()
    return ret_df[['title', 'genres', 'overview']]

In [5]:
ten_k_over = df_overview_clean(ten_k)

In [6]:
classes = ['Action', 'Adventure', 'Animation', 'Comedy', 'Crime', 'Documentary',
           'Drama', 'Family', 'Fantasy', 'Foreign', 'History', 'Horror', 'Music',
            'Mystery', 'Romance', 'Science Fiction', 'TV Movie', 'Thriller', 'War', 'Western']

In [7]:
def genre_extract(genre_dic):
    ret_genres = []
    for dic in genre_dic:
        ret_genres.append(dic['name'])
    return ret_genres

def assign_genre_vector(row_genres, all_classes):
    ret_list = []
    genre_list = genre_extract(row_genres)
    for genre in all_classes:
        if genre in genre_list:
            ret_list.append(1)
        else:
            ret_list.append(0)
    return ret_list           

In [8]:
ten_k_vec = pd.DataFrame(columns=['title', 'genres', 'overview', 'genre_vec', 'all_genres'])
for row in zip(ten_k_over['title'], ten_k_over['genres'], ten_k_over['overview']):
    genre_vec = assign_genre_vector(row[1], classes)
    genres = genre_extract(row[1])
    
    #had to throw this in for the vectorization process-later
    overview = row[2]
    if row[2] == None or not row[2]:
        overview = 'a'
    ten_k_vec = ten_k_vec.append({'title':row[0], 'genres':row[1], 'overview':overview, 'genre_vec':genre_vec, 'all_genres':genres, 'sin_gen':genres[0]}, ignore_index=True)

In [9]:
#set our vectorization
vectorizer = CountVectorizer(stop_words=stopwords.words("english"),
    max_df = 0.9, 
    min_df = 5, 
    dtype=np.float32 )

corpus = ten_k_vec['overview'].values
X = vectorizer.fit_transform(corpus).toarray()
ys = MultiLabelBinarizer().fit_transform(ten_k_vec['all_genres'])
#ys = five_k_vec['sin_gen']

In [52]:
uni_classes = [list(x) for x in set(tuple(x) for x in ten_k_vec['all_genres'].tolist())]

Basic Nueral Network

In [23]:
X_train,X_test,y_train,y_test = train_test_split(X,ys,test_size=0.3)

In [54]:
# smaller batch size means noisier gradient, but more updates per epoch
batch_size = 512
num_classes = len(classes)
# number of iterations over the complete training data
epochs = 1000

In [55]:
print(X_train.shape[0], 'train samples', X_train.shape[1], 'features')
print(X_test.shape[0], 'test samples')

7000 train samples 7950 features
3000 test samples


In [57]:
model = None

model = Sequential()
#input layer
model.add(Dense(64, activation='relu', input_dim=7950))
#hidden layers
model.add(Dense(64, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(64, activation='relu'))
#output layer
model.add(Dense(num_classes, activation='softmax'))
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_39 (Dense)             (None, 64)                508864    
_________________________________________________________________
dense_40 (Dense)             (None, 64)                4160      
_________________________________________________________________
dense_41 (Dense)             (None, 64)                4160      
_________________________________________________________________
dense_42 (Dense)             (None, 64)                4160      
_________________________________________________________________
dense_43 (Dense)             (None, 1329)              86385     
Total params: 607,729
Trainable params: 607,729
Non-trainable params: 0
_________________________________________________________________


In [58]:
sgd = SGD(lr=0.01, momentum=0.9)
model.compile(loss='categorical_crossentropy',
              optimizer=sgd,
              metrics=['accuracy'])

In [59]:
history = model.fit(X_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=0,
                    validation_data=(X_test, y_test))

score = model.evaluate(X_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

ValueError: Error when checking model target: expected dense_43 to have shape (None, 1329) but got array with shape (7000, 20)

In [ ]:
plt.plot(history.history['acc'])
plt.xlabel("epoch")
plt.ylabel("accuracy")

In [ ]:
y_pred_proba = model.predict(X_test)
y_pred_test = np.zeros(y_pred_proba.shape)
y_pred_test[y_pred_proba > 0.5] = 1
from sklearn.metrics import classification_report
print(classification_report(y_test, y_pred_test, target_names = classes))